In [1]:
from config import *
import utilities as utils

import importlib

# importlib.reload(utils)

year_files_dict = utils.get_all_filenames()

In [2]:
import gensim
import nltk
import pandas as pd
import re
import itertools

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
stopwords_list = nltk.corpus.stopwords.words('english')

import json

### Get all tokenized sentences

In [ ]:
all_words = list()
tokenized_doc = dict()

for fname in itertools.chain.from_iterable(year_files_dict.values()):
    tokenized_sentences = list()
    data = utils.load_data(fname)
    count = 0
    for art in data['Items']:
        text = ''
        if art['data']['language'] == 'en':
            if any(subject in art['data']['subjects'] for subject in [US_CODE]):
                text = art['data']['body']

        if not text:
            continue
        count += 1

        text = nltk.sent_tokenize(text)
        text = [sentence.replace('\n', ' ') for sentence in text]
        doc_words = []
        for sentence in text:
            tokenized_sent = list()
            # Only keeping alphabets and replacing everything else with spaces
            sentence = re.sub(r'[^A-Za-z]+', ' ', sentence)
            sentence = sentence.lower()
            sent_words = nltk.word_tokenize(sentence)
            sent_words = [word for word in sent_words if ((len(word) > 1) and (len(word) < 20))]
            sent_words = [word for word in sent_words if word not in stopwords_list]
            for word in sent_words:
                doc_words.append(word)
                all_words.append(word)
                tokenized_sent.append(word)
            tokenized_sentences.append(tokenized_sent)

        tokenized_doc[art['data']['id']] = doc_words
    print(fname, 'Done! Article Count:', count, 'Storing in a json now')
    with open(TEMP_PATH + '/%s.txt' % fname[-15:-9], 'w') as outfile:
        json.dump(tokenized_sentences, outfile)
    del data

In [ ]:
del tokenized_doc

In [ ]:
len(all_words)

In [46]:
import glob
files = glob.glob(TEMP_PATH + '/2019*')

In [ ]:
tokenized_sentences = []
for f in files:
    f = open(f)
    lines = f.readlines()
    tokenized_sentences.extend(eval(lines[0]))
    print(f, 'done!')

### Gensim Model - To get the similar words to our three themes

In [14]:
from gensim.models import Word2Vec

In [17]:
Word2Vec?

Init signature:
Word2Vec(
    sentences=None,
    corpus_file=None,
    vector_size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    epochs=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    comment=None,
    max_final_vocab=None,
)
Docstring:     
Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.

Warnings
--------
This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
such as lambda functions etc.
Init docstring:
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:

In [ ]:
%env PYTHONHASHSEED = 1
model = Word2Vec(tokenized_sentences)
model.save("all_reuters_news_articles_2019.w2v")

In [56]:
pd.DataFrame(model.wv.most_similar('volatile'), columns=['Word', 'Similarity'])

,Word,Similarity
0,subdued,0.593277
1,fluctuating,0.591763
2,depressed,0.578572
3,choppy,0.576144
4,compressed,0.562198
5,rangebound,0.549121
6,thin,0.547195
7,spiking,0.546485
8,volatility,0.545142
9,stabilised,0.541073


In [57]:
pd.DataFrame(model.wv.most_similar('risk'), columns=['Word', 'Similarity'])

,Word,Similarity
0,risks,0.610763
1,likelihood,0.555918
2,volatility,0.546834
3,strengths,0.535214
4,conversely,0.519471
5,exposure,0.515775
6,probability,0.503422
7,probabilities,0.498103
8,exposures,0.497595
9,eeh,0.488991


In [58]:
model.wv.most_similar('volatility')

[('volatilities', 0.7111401557922363),
 ('vols', 0.6137344241142273),
 ('vol', 0.6022745370864868),
 ('correlation', 0.5784175395965576),
 ('fluctuations', 0.5549601316452026),
 ('risk', 0.5468342304229736),
 ('headwind', 0.5459719896316528),
 ('volatile', 0.5451425313949585),
 ('indecision', 0.5359459519386292),
 ('skew', 0.534342885017395)]

In [59]:
model.wv.most_similar('gyrations')

[('selloff', 0.5864651799201965),
 ('pullback', 0.5662065148353577),
 ('rout', 0.5490662455558777),
 ('steadying', 0.5461956262588501),
 ('rallying', 0.5403254628181458),
 ('bounce', 0.5326281785964966),
 ('correction', 0.5237931609153748),
 ('reaction', 0.5092394351959229),
 ('pullbacks', 0.5087333917617798),
 ('rally', 0.5045714974403381)]

In [60]:
model.wv.most_similar('inflation')

[('cpi', 0.6701036095619202),
 ('gdp', 0.6695371866226196),
 ('pmi', 0.5775833129882812),
 ('unemployment', 0.5531595945358276),
 ('ppi', 0.5304327607154846),
 ('easing', 0.5087022185325623),
 ('jobless', 0.5005810260772705),
 ('employment', 0.4985373020172119),
 ('policymakers', 0.4956326484680176),
 ('deflation', 0.4950767457485199)]

In [61]:
model.wv.most_similar('gdp')

[('inflation', 0.6695373058319092),
 ('cpi', 0.659629225730896),
 ('pmi', 0.6340224742889404),
 ('deficit', 0.5387327671051025),
 ('ppi', 0.5227527022361755),
 ('surplus', 0.5204976797103882),
 ('ifo', 0.5201418995857239),
 ('cenbank', 0.5117142200469971),
 ('pmis', 0.5026580095291138),
 ('manufacturing', 0.498676598072052)]

In [ ]:
# TODO
# IF we are going down this path, parallel processing on the grid is probably the best option.
# Or increasing memory allocation for notebook might help too. 